In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from scipy.misc import imsave
from skimage.draw import circle
from scipy.ndimage.filters import gaussian_filter
import os

In [2]:
annotations = loadmat('labels/0001.mat')
print annotations.keys()

['nframes', 'dimensions', 'pose', '__header__', 'visibility', '__globals__', 'y', 'train', 'bbox', 'action', 'x', '__version__']


# Analogy Network Preprocessing

## Stacked Joints (1 image for 13 joints)

In [4]:
radius = 5
gaussian_sigma = 5

directory = 'labels/'
# Create this out_directory
out_directory = 'squats_labels/'

for filename in os.listdir(directory):
    annotations = loadmat(directory + filename)
    visibility = annotations['visibility']
    if annotations['action'] == 'squat':
        # Create stack of zero initialized images for joints
        vol = np.zeros(annotations['dimensions'][0])
        X = annotations['x']
        Y = annotations['y']
        for i in range(vol.shape[2]):
            for j in range(visibility.shape[1]):
    #             if visibility[i][j] == 1:
                coords = circle(X[i][j],Y[i][j],radius)
                y_coords = coords[1]
                x_coords = coords[0]

                # Clip circle coordinate values, keep everything in bounds
                y_coords[y_coords > vol.shape[0]-1] = vol.shape[0] - 1
                x_coords[x_coords > vol.shape[1]-1] = vol.shape[1] - 1
                y_coords[y_coords < 0] = 0
                x_coords[x_coords < 0] = 0

                # Draw circles
                vol[coords[1],coords[0],i] = 255
            vol[:,:,i] = gaussian_filter(vol[:,:,i],gaussian_sigma)
        for i in range(vol.shape[2]):
            folder = filename.split('.')[0] + '/'
            if not os.path.exists(out_directory  + folder):
                os.makedirs(out_directory + folder)
            imsave(out_directory + folder + '%04d'%(i) + '.jpg', vol[:,:,i])

## Individual joints (13 images, 1 per joint)

In [ ]:
radius = 5
gaussian_sigma = 5

directory = 'labels/'
# Create this out_directory
out_directory = 'squats_labels_multiple/'

for filename in os.listdir(directory):
    annotations = loadmat(directory + filename)
    visibility = annotations['visibility']
    if annotations['action'] == 'squat':
        # Create stack of zero initialized images for joints
        vol = np.zeros(np.concatenate((annotations['dimensions'][0],np.array([13]))))
        X = annotations['x']
        Y = annotations['y']
        for i in range(vol.shape[2]):
            for j in range(visibility.shape[1]):
    #             if visibility[i][j] == 1:
                coords = circle(X[i][j],Y[i][j],radius)
                y_coords = coords[1]
                x_coords = coords[0]

                # Clip circle coordinate values, keep everything in bounds
                y_coords[y_coords > vol.shape[0]-1] = vol.shape[0] - 1
                x_coords[x_coords > vol.shape[1]-1] = vol.shape[1] - 1
                y_coords[y_coords < 0] = 0
                x_coords[x_coords < 0] = 0

                # Draw circles
                vol[coords[1],coords[0],i,j] = 255
                vol[:,:,i,j] = gaussian_filter(vol[:,:,i,j],gaussian_sigma)
        for i in range(vol.shape[2]):
            for j in range(visibility.shape[1]):
                folder = filename.split('.')[0] + '/'
                if not os.path.exists(out_directory  + folder  + '%04d'%(i) + '/'):
                    os.makedirs(out_directory + folder + '%04d'%(i) + '/')
                imsave(out_directory + folder + '%04d'%(i) + '/' + str(j) + '.jpg', vol[:,:,i,j])